In [287]:
import numpy as np
import pandas as pd
import math
from datetime import datetime, timedelta
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, precision_score
import pickle

In [288]:
insulin_data = pd.DataFrame(pd.read_csv('insulinData.csv'))

C:\Users\nisha\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (13,43) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [289]:
insulin_data_p2 = pd.DataFrame(pd.read_csv('InsulinAndMealIntake670GPatient3.csv'))

C:\Users\nisha\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [290]:
cgm_data = pd.DataFrame(pd.read_csv('CGMData.csv'))

C:\Users\nisha\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [291]:
cgm_data_p2 = pd.DataFrame(pd.read_csv('CGMData670GPatient3.csv'))

In [292]:
match_time = []
match_date = []
for i in range(len(insulin_data)):
    if(not(math.isnan(insulin_data.loc[i, 'BWZ Carb Input (grams)']))):
        match_date.append(insulin_data.loc[i, 'Date'])
        match_time.append(insulin_data.loc[i, 'Time'])

In [293]:
match_time_p2 = []
match_date_p2 = []
for i in range(len(insulin_data_p2)):
    if(not(math.isnan(insulin_data_p2.loc[i, 'BWZ Carb Input (grams)']))):
        match_date_p2.append(insulin_data_p2.loc[i, 'Date'])
        match_time_p2.append(insulin_data_p2.loc[i, 'Time'])

In [294]:
dtl = []
for i in range(len(match_date)):
    dtl.append(str(match_date[i]) + ' ' + str(match_time[i]))

In [295]:
dtl_p2 = []
for i in range(len(match_date_p2)):
    dtl_p2.append(str(match_date_p2[i]) + ' ' + str(match_time_p2[i]))

In [296]:
insulin_data = insulin_data.iloc[::-1]

In [297]:
insulin_data_p2 = insulin_data_p2.iloc[::-1]

In [298]:
cgm_data = cgm_data[::-1]

In [299]:
cgm_data_p2 = cgm_data_p2.iloc[::-1]

In [300]:
for i in range(len(dtl)):
    dtl[i] = datetime.strptime(str(match_date[i]) + ' ' + str(match_time[i]), '%m/%d/%Y %H:%M:%S')


In [301]:
for i in range(len(dtl_p2)):
    dtl_p2[i] = datetime.strptime(str(match_date_p2[i]) + ' ' + str(match_time_p2[i]), '%d-%m-%Y %H:%M:%S')

In [302]:
dtl = dtl[::-1]

In [303]:
dtl_p2 = dtl_p2[::-1]

In [304]:
meal_data_dt = []
for i in range(len(dtl)-1):
    if((dtl[i] + timedelta(hours = 2)) < dtl[i+1]):
        meal_data_dt.append(dtl[i])
    if((dtl[i] + timedelta(hours = 2)) == dtl[i+1]):
        meal_data_dt.append(dtl[i+1] + timedelta(hours = -0.5))
    

In [305]:
meal_data_dt_p2 = []
for i in range(len(dtl_p2)-1):
    if((dtl_p2[i] + timedelta(hours = 2)) < dtl_p2[i+1]):
        meal_data_dt_p2.append(dtl_p2[i])
    if((dtl_p2[i] + timedelta(hours = 2)) == dtl_p2[i+1]):
        meal_data_dt_p2.append(dtl_p2[i+1] + timedelta(hours = -0.5))

In [306]:
no_meal_data_dt = []
for i in range(len(dtl)-1):
    if((dtl[i] + timedelta(hours = 4)) < dtl[i+1]):
        no_meal_data_dt.append(dtl[i] + timedelta(hours = 2))

In [307]:
no_meal_data_dt_p2 = []
for i in range(len(dtl_p2)-1):
    if((dtl_p2[i] + timedelta(hours = 4)) < dtl_p2[i+1]):
        no_meal_data_dt_p2.append(dtl_p2[i] + timedelta(hours = 2))

In [308]:
cgm_data_dt = []
for i in range(len(cgm_data)):
    cgm_data_dt.append(datetime.strptime(str(cgm_data.loc[i, 'Date']) + ' ' + str(cgm_data.loc[i, 'Time']), '%m/%d/%Y %H:%M:%S'))

In [309]:
cgm_data_dt_p2 = []
for i in range(len(cgm_data_p2)):
    cgm_data_dt_p2.append(datetime.strptime(str(cgm_data_p2.loc[i, 'Date']) + ' ' + str(cgm_data_p2.loc[i, 'Time']), '%d-%m-%Y %H:%M:%S'))

# meal data of cgm


In [310]:
cgm_tm_match_meal = []
for i in range(len(meal_data_dt)):
    for j in range(len(cgm_data_dt)):
        if(cgm_data_dt[j] < meal_data_dt[i] + timedelta(minutes = 5)):
            cgm_tm_match_meal.append((cgm_data_dt[j]))
            break

In [311]:
cgm_tm_match_meal_p2 = []
for i in range(len(meal_data_dt_p2)):
    for j in range(len(cgm_data_dt_p2)):
        if(cgm_data_dt_p2[j] < meal_data_dt_p2[i] + timedelta(minutes = 5)):
            cgm_tm_match_meal_p2.append((cgm_data_dt_p2[j]))
            break

# no meal data of CGM


In [312]:
cgm_tm_match_no_meal = []
for i in range(len(no_meal_data_dt)):
    for j in range(len(cgm_data_dt)):
        if(cgm_data_dt[j] < no_meal_data_dt[i] + timedelta(minutes = 5)):
            cgm_tm_match_no_meal.append((cgm_data_dt[j]))
            break

In [313]:
cgm_tm_match_no_meal_p2 = []
for i in range(len(no_meal_data_dt_p2)):
    for j in range(len(cgm_data_dt_p2)):
        if(cgm_data_dt_p2[j] < no_meal_data_dt_p2[i] + timedelta(minutes = 5)):
            cgm_tm_match_no_meal_p2.append((cgm_data_dt_p2[j]))
            break

In [314]:
cgm_data['timestamp'] = cgm_data_dt[::-1]

In [315]:
cgm_data_p2['timestamp'] = cgm_data_dt_p2[::-1]

# Meal data cgm values

In [316]:
temp_index_list = []
for i in cgm_tm_match_meal:
    temp_index_list.append(cgm_data[cgm_data['timestamp'] == i].index[0])

In [317]:
temp_index_list_p2 = []
for i in cgm_tm_match_meal_p2:
    temp_index_list_p2.append(cgm_data_p2[cgm_data_p2['timestamp'] == i].index[0])

In [318]:
cgm_value_stretch_meal = []
for i in range(1,len(temp_index_list)):
    cgm_value_one_stretch_meal = []
    for j in range(30):
        cgm_value_one_stretch_meal.append(cgm_data.loc[(temp_index_list[i] + 6 - j),'Sensor Glucose (mg/dL)'])
    if(not(np.isnan(cgm_value_one_stretch_meal).all())):
        cgm_value_stretch_meal.append(cgm_value_one_stretch_meal)


In [319]:
cgm_value_stretch_meal_p2 = []
for i in range(1,len(temp_index_list_p2)):
    cgm_value_one_stretch_meal_p2 = []
    for j in range(30):
        cgm_value_one_stretch_meal_p2.append(cgm_data_p2.loc[(temp_index_list_p2[i] + 6 - j),'Sensor Glucose (mg/dL)'])
    if(not(np.isnan(cgm_value_one_stretch_meal_p2).all())):
        cgm_value_stretch_meal_p2.append(cgm_value_one_stretch_meal_p2)

In [320]:
for i in range(len(cgm_value_stretch_meal)):
    for j in range(len(cgm_value_stretch_meal[i])):
        if(np.isnan(cgm_value_stretch_meal[i][j])):
            cgm_value_stretch_meal[i][j] = int(np.nanmean(cgm_value_stretch_meal[i]))

In [321]:
for i in range(len(cgm_value_stretch_meal_p2)):
    for j in range(len(cgm_value_stretch_meal_p2[i])):
        if(np.isnan(cgm_value_stretch_meal_p2[i][j])):
            cgm_value_stretch_meal_p2[i][j] = int(np.nanmean(cgm_value_stretch_meal_p2[i]))

In [322]:
cgm_value_stretch_meal_df = pd.DataFrame(cgm_value_stretch_meal)

In [323]:
cgm_value_stretch_meal_df_p2 = pd.DataFrame(cgm_value_stretch_meal_p2)

# no meal data cgm values

In [324]:
temp_index_no_meal_list = []
for i in cgm_tm_match_no_meal:
    temp_index_no_meal_list.append(cgm_data[cgm_data['timestamp'] == i].index[0])

In [325]:
temp_index_no_meal_list_p2 = []
for i in cgm_tm_match_no_meal_p2:
    temp_index_no_meal_list_p2.append(cgm_data_p2[cgm_data_p2['timestamp'] == i].index[0])

In [326]:
cgm_value_stretch_no_meal = []
for i in range(len(temp_index_no_meal_list)):
    cgm_value_one_stretch_no_meal = []
    for j in range(24):
        cgm_value_one_stretch_no_meal.append(cgm_data.loc[(temp_index_no_meal_list[i] - j),'Sensor Glucose (mg/dL)'])
    if(not(np.isnan(cgm_value_one_stretch_no_meal).all())):
        cgm_value_stretch_no_meal.append(cgm_value_one_stretch_no_meal)

In [327]:
cgm_value_stretch_no_meal_p2 = []
for i in range(len(temp_index_no_meal_list_p2)):
    cgm_value_one_stretch_no_meal_p2 = []
    for j in range(24):
        cgm_value_one_stretch_no_meal_p2.append(cgm_data_p2.loc[(temp_index_no_meal_list_p2[i] - j),'Sensor Glucose (mg/dL)'])
    if(not(np.isnan(cgm_value_one_stretch_no_meal_p2).all())):
        cgm_value_stretch_no_meal_p2.append(cgm_value_one_stretch_no_meal_p2)

In [328]:
for i in range(len(cgm_value_stretch_no_meal)):
    for j in range(len(cgm_value_stretch_no_meal[i])):
        if(np.isnan(cgm_value_stretch_no_meal[i][j])):
            cgm_value_stretch_no_meal[i][j] = (np.nanmean(cgm_value_stretch_no_meal[i]))

In [329]:
for i in range(len(cgm_value_stretch_no_meal_p2)):
    for j in range(len(cgm_value_stretch_no_meal_p2[i])):
        if(np.isnan(cgm_value_stretch_no_meal_p2[i][j])):
            cgm_value_stretch_no_meal_p2[i][j] = (np.nanmean(cgm_value_stretch_no_meal_p2[i]))

In [330]:
cgm_value_stretch_no_meal_df = pd.DataFrame(cgm_value_stretch_no_meal)

In [331]:
cgm_value_stretch_no_meal_df_p2 = pd.DataFrame(cgm_value_stretch_no_meal_p2)

# Feature Engineering

# CGM Velocity, tmax

In [332]:
cgm_velocity_meal_p1 = np.diff(cgm_value_stretch_meal_df)

In [333]:
cgm_velocity_meal_p2 = np.diff(cgm_value_stretch_meal_df_p2)

In [334]:
cgm_velocity_no_meal_p1 = np.diff(cgm_value_stretch_no_meal_df)

In [335]:
cgm_velocity_no_meal_p2 = np.diff(cgm_value_stretch_no_meal_df_p2)

In [336]:
max_cgm_velocity_meal_p1 = []
for i in range(len(cgm_velocity_meal_p1)):
    max_cgm_velocity_meal_p1.append([max(cgm_velocity_meal_p1[i]), np.argmax(cgm_velocity_meal_p1[i])*5])

In [337]:
max_cgm_velocity_meal_p2 = []
for i in range(len(cgm_velocity_meal_p2)):
    max_cgm_velocity_meal_p2.append([max(cgm_velocity_meal_p2[i]), np.argmax(cgm_velocity_meal_p2[i])*5])

In [338]:
max_cgm_velocity_no_meal_p1 = []
for i in range(len(cgm_velocity_no_meal_p1)):
    max_cgm_velocity_no_meal_p1.append([max(cgm_velocity_no_meal_p1[i]), np.argmax(cgm_velocity_no_meal_p1[i])*5])

In [339]:
max_cgm_velocity_no_meal_p2 = []
for i in range(len(cgm_velocity_no_meal_p2)):
    max_cgm_velocity_no_meal_p2.append([max(cgm_velocity_no_meal_p2[i]), np.argmax(cgm_velocity_no_meal_p2[i])*5])

# CGMmax - CGMmin

In [340]:
cgm_range_meal_p1 = cgm_value_stretch_meal_df.max(axis = 1) - cgm_value_stretch_meal_df.min(axis = 1)

In [341]:
cgm_range_no_meal_p1 = cgm_value_stretch_no_meal_df.max(axis = 1) - cgm_value_stretch_no_meal_df.min(axis = 1)

In [342]:
cgm_range_meal_p2 = cgm_value_stretch_meal_df_p2.max(axis = 1) - cgm_value_stretch_meal_df_p2.min(axis = 1)

In [343]:
cgm_range_no_meal_p2 = cgm_value_stretch_no_meal_df_p2.max(axis = 1) - cgm_value_stretch_no_meal_df_p2.min(axis = 1)

# Important duration

In [344]:
imp_duration_meal_p1 = (cgm_value_stretch_meal_df.idxmax(axis = 1))*5

In [345]:
imp_duration_no_meal_p1 = (cgm_value_stretch_no_meal_df.idxmax(axis = 1))*5

In [346]:
imp_duration_meal_p2 = (cgm_value_stretch_meal_df_p2.idxmax(axis = 1))*5

In [347]:
imp_duration_no_meal_p2 = (cgm_value_stretch_no_meal_df_p2.idxmax(axis = 1))*5

# FFT

In [348]:
fft_meal_p1 = []
temp1 = []
#for i in range(len(cgm_value_stretch_meal_df)):
temp1 = abs(np.fft.fft(cgm_value_stretch_meal_df)) #[i:i+1])).flatten()
for i in range(len(temp1)):
    temp1[i].sort()
    fft_meal_p1.append([temp1[i][-2], temp1[i][-4]])

In [349]:
fft_no_meal_p1 = []
temp1 = []
#for i in range(len(cgm_value_stretch_meal_df)):
temp1 = abs(np.fft.fft(cgm_value_stretch_no_meal_df)) #[i:i+1])).flatten()
for i in range(len(temp1)):
    temp1[i].sort()
    fft_no_meal_p1.append([temp1[i][-2], temp1[i][-4]])

In [350]:
fft_meal_p2 = []
temp1 = []
#for i in range(len(cgm_value_stretch_meal_df)):
temp1 = abs(np.fft.fft(cgm_value_stretch_meal_df_p2)) #[i:i+1])).flatten()
for i in range(len(temp1)):
    temp1[i].sort()
    fft_meal_p2.append([temp1[i][-2], temp1[i][-4]])

In [351]:
fft_no_meal_p2 = []
temp1 = []
#for i in range(len(cgm_value_stretch_meal_df)):
temp1 = abs(np.fft.fft(cgm_value_stretch_no_meal_df_p2)) #[i:i+1])).flatten()
for i in range(len(temp1)):
    temp1[i].sort()
    fft_no_meal_p2.append([temp1[i][-2], temp1[i][-4]])

# windowed mean

In [352]:
win_mean_meal_p1 = []
win_mean_meal_p1.append([cgm_value_stretch_meal_df.rolling(3, win_type='triang', axis = 1).mean()[8], cgm_value_stretch_meal_df.rolling(3, win_type='triang', axis = 1).mean()[11], cgm_value_stretch_meal_df.rolling(3, win_type='triang', axis = 1).mean()[14], cgm_value_stretch_meal_df.rolling(3, win_type='triang', axis = 1).mean()[17], cgm_value_stretch_meal_df.rolling(3, win_type='triang', axis = 1).mean()[20]])

In [353]:
win_mean_no_meal_p1 = []
win_mean_no_meal_p1.append([cgm_value_stretch_no_meal_df.rolling(3, win_type='triang', axis = 1).mean()[8], cgm_value_stretch_no_meal_df.rolling(3, win_type='triang', axis = 1).mean()[11], cgm_value_stretch_no_meal_df.rolling(3, win_type='triang', axis = 1).mean()[14], cgm_value_stretch_no_meal_df.rolling(3, win_type='triang', axis = 1).mean()[17], cgm_value_stretch_no_meal_df.rolling(3, win_type='triang', axis = 1).mean()[20]])

In [354]:
win_mean_meal_p2 = []
win_mean_meal_p2.append([cgm_value_stretch_meal_df_p2.rolling(3, win_type='triang', axis = 1).mean()[8], cgm_value_stretch_meal_df_p2.rolling(3, win_type='triang', axis = 1).mean()[11], cgm_value_stretch_meal_df_p2.rolling(3, win_type='triang', axis = 1).mean()[14], cgm_value_stretch_meal_df_p2.rolling(3, win_type='triang', axis = 1).mean()[17], cgm_value_stretch_meal_df_p2.rolling(3, win_type='triang', axis = 1).mean()[20]])

In [355]:
win_mean_no_meal_p2 = []
win_mean_no_meal_p2.append([cgm_value_stretch_no_meal_df_p2.rolling(3, win_type='triang', axis = 1).mean()[8], cgm_value_stretch_no_meal_df_p2.rolling(3, win_type='triang', axis = 1).mean()[11], cgm_value_stretch_no_meal_df_p2.rolling(3, win_type='triang', axis = 1).mean()[14], cgm_value_stretch_no_meal_df_p2.rolling(3, win_type='triang', axis = 1).mean()[17], cgm_value_stretch_no_meal_df_p2.rolling(3, win_type='triang', axis = 1).mean()[20]])

In [356]:
feat_win_meal_p2 = []
for j in range(5):
    temp = []
    for i in range(len(win_mean_meal_p2[0][j])):
        temp.append(win_mean_meal_p2[0][j][i])
    feat_win_meal_p2.append(temp)
feat_win_meal_p2 = np.array(feat_win_meal_p2).T

In [357]:
feat_win_meal_p1 = []
for j in range(5):
    temp = []
    for i in range(len(win_mean_meal_p1[0][j])):
        temp.append(win_mean_meal_p1[0][j][i])
    feat_win_meal_p1.append(temp)
feat_win_meal_p1 = np.array(feat_win_meal_p1).T

In [358]:
feat_win_no_meal_p1 = []
for j in range(5):
    temp = []
    for i in range(len(win_mean_no_meal_p1[0][j])):
        temp.append(win_mean_no_meal_p1[0][j][i])
    feat_win_no_meal_p1.append(temp)
feat_win_no_meal_p1 = np.array(feat_win_no_meal_p1).T

In [359]:
feat_win_no_meal_p2 = []
for j in range(5):
    temp = []
    for i in range(len(win_mean_no_meal_p2[0][j])):
        temp.append(win_mean_no_meal_p2[0][j][i])
    feat_win_no_meal_p2.append(temp)
feat_win_no_meal_p2 = np.array(feat_win_no_meal_p2).T

# Creating feature matrix

In [360]:
'''max_cgm_velocity_meal_p1 = []
for i in range(len(cgm_velocity_meal_p1)):
    max_cgm_velocity_meal_p1.append([max(cgm_velocity_meal_p1[i]), np.argmax(cgm_velocity_meal_p1[i])*5])'''
final_feature_matrix = []
feature_matrix = []
feature_matrix.append(max_cgm_velocity_meal_p1)
final_feature_matrix.append(feature_matrix[0])
final_feature_matrix = final_feature_matrix[0]

In [361]:
'''max_cgm_velocity_no_meal_p1 = []
for i in range(len(cgm_velocity_no_meal_p1)):
    max_cgm_velocity_no_meal_p1.append([max(cgm_velocity_no_meal_p1[i]), np.argmax(cgm_velocity_no_meal_p1[i])*5])'''
final_feature_matrix_no_meal_p1 = []
feature_matrix_no_meal_p1 = []
feature_matrix_no_meal_p1.append(max_cgm_velocity_no_meal_p1)
final_feature_matrix_no_meal_p1.append(feature_matrix_no_meal_p1[0])
final_feature_matrix_no_meal_p1 = final_feature_matrix_no_meal_p1[0]

In [362]:
'''max_cgm_velocity_meal_p2 = []
for i in range(len(cgm_velocity_meal_p2)):
    max_cgm_velocity_meal_p2.append([max(cgm_velocity_meal_p2[i]), np.argmax(cgm_velocity_meal_p2[i])*5])'''
final_feature_matrix_meal_p2 = []
feature_matrix_meal_p2 = []
feature_matrix_meal_p2.append(max_cgm_velocity_meal_p2)
final_feature_matrix_meal_p2.append(feature_matrix_meal_p2[0])
final_feature_matrix_meal_p2 = final_feature_matrix_meal_p2[0]

In [363]:
'''max_cgm_velocity_no_meal_p2 = []
for i in range(len(cgm_velocity_no_meal_p2)):
    max_cgm_velocity_no_meal_p2.append([max(cgm_velocity_no_meal_p2[i]), np.argmax(cgm_velocity_no_meal_p2[i])*5])'''
final_feature_matrix_no_meal_p2 = []
feature_matrix_no_meal_p2 = []
feature_matrix_no_meal_p2.append(max_cgm_velocity_no_meal_p2)
final_feature_matrix_no_meal_p2.append(feature_matrix_no_meal_p2[0])
final_feature_matrix_no_meal_p2 = final_feature_matrix_no_meal_p2[0]

In [364]:
for i in range(len(final_feature_matrix)):
    final_feature_matrix[i].append(cgm_range_meal_p1[i])

In [365]:
for i in range(len(final_feature_matrix_no_meal_p1)):
    final_feature_matrix_no_meal_p1[i].append(cgm_range_no_meal_p1[i])

In [366]:
for i in range(len(final_feature_matrix_meal_p2)):
    final_feature_matrix_meal_p2[i].append(cgm_range_meal_p2[i])

In [367]:
for i in range(len(final_feature_matrix_no_meal_p2)):
    final_feature_matrix_no_meal_p2[i].append(cgm_range_no_meal_p2[i])

# important duration for test

In [368]:
for i in range(len(final_feature_matrix_no_meal_p1)):
    final_feature_matrix_no_meal_p1[i].append(imp_duration_no_meal_p1[i])

In [369]:
for i in range(len(final_feature_matrix)):
    final_feature_matrix[i].append(imp_duration_meal_p1[i])

In [370]:
for i in range(len(final_feature_matrix_meal_p2)):
    final_feature_matrix_meal_p2[i].append(imp_duration_meal_p2[i])

In [371]:
for i in range(len(final_feature_matrix_no_meal_p2)):
    final_feature_matrix_no_meal_p2[i].append(imp_duration_no_meal_p2[i])

In [372]:
for i in range(len(final_feature_matrix)):
    final_feature_matrix[i].append(fft_meal_p1[i][0])
    final_feature_matrix[i].append(fft_meal_p1[i][1])

In [373]:
for i in range(len(final_feature_matrix_no_meal_p1)):
    final_feature_matrix_no_meal_p1[i].append(fft_no_meal_p1[i][0])
    final_feature_matrix_no_meal_p1[i].append(fft_no_meal_p1[i][1])

In [374]:
for i in range(len(final_feature_matrix_meal_p2)):
    final_feature_matrix_meal_p2[i].append(fft_meal_p2[i][0])
    final_feature_matrix_meal_p2[i].append(fft_meal_p2[i][1])

In [375]:
for i in range(len(final_feature_matrix_no_meal_p2)):
    final_feature_matrix_no_meal_p2[i].append(fft_no_meal_p2[i][0])
    final_feature_matrix_no_meal_p2[i].append(fft_no_meal_p2[i][1])

In [376]:
for i in range(len(final_feature_matrix)):
    for j in range(5):
        final_feature_matrix[i].append(feat_win_meal_p1[i][j])

In [377]:
for i in range(len(final_feature_matrix_no_meal_p1)):
    for j in range(5):
        final_feature_matrix_no_meal_p1[i].append(feat_win_no_meal_p1[i][j])

In [378]:
for i in range(len(final_feature_matrix_no_meal_p2)):
    for j in range(5):
        final_feature_matrix_no_meal_p2[i].append(feat_win_no_meal_p2[i][j])

In [379]:
for i in range(len(final_feature_matrix_meal_p2)):
    for j in range(5):
        final_feature_matrix_meal_p2[i].append(feat_win_meal_p2[i][j])

In [380]:
final_feature_matrix = (np.array(final_feature_matrix).T).T

In [381]:
final_feature_matrix_no_meal_p1 = (np.array(final_feature_matrix_no_meal_p1).T).T

In [382]:
final_feature_matrix_meal_p2 = (np.array(final_feature_matrix_meal_p2).T).T

In [383]:
final_feature_matrix_no_meal_p2 = (np.array(final_feature_matrix_no_meal_p2).T).T

In [384]:
final_feature_df = pd.DataFrame(final_feature_matrix, columns=['Max CGM velocity', 'time at max vel', 'CGM max-min', 'Important duration', 'FFT1', 'FFT2', 'Win_mean1', 'Win_mean2', 'Win_mean3', 'Win_mean4', 'Win_mean5'])

In [385]:
final_feature_df_no_meal_p1 = pd.DataFrame(final_feature_matrix_no_meal_p1, columns=['Max CGM velocity', 'time at max vel', 'CGM max-min', 'Important duration', 'FFT1', 'FFT2', 'Win_mean1', 'Win_mean2', 'Win_mean3', 'Win_mean4', 'Win_mean5'])

In [386]:
final_feature_df_meal_p2 = pd.DataFrame(final_feature_matrix_meal_p2, columns=['Max CGM velocity', 'time at max vel', 'CGM max-min', 'Important duration', 'FFT1', 'FFT2', 'Win_mean1', 'Win_mean2', 'Win_mean3', 'Win_mean4', 'Win_mean5'])

In [387]:
final_feature_df_no_meal_p2 = pd.DataFrame(final_feature_matrix_no_meal_p2, columns=['Max CGM velocity', 'time at max vel', 'CGM max-min', 'Important duration', 'FFT1', 'FFT2', 'Win_mean1', 'Win_mean2', 'Win_mean3', 'Win_mean4', 'Win_mean5'])

In [388]:
final_feature_df_no_meal_p1['meal'] = 0

In [389]:
final_feature_df['meal'] = 1

In [390]:
final_feature_df_no_meal_p2['meal'] = 0

In [391]:
final_feature_df_meal_p2['meal'] = 1

# training data 

In [392]:
final_data_p1 = pd.concat([final_feature_df[0:200], final_feature_df_no_meal_p1[0:200]])

# testing data

In [393]:
test_final_data_p2 = pd.concat([final_feature_df_meal_p2, final_feature_df_no_meal_p2])

In [394]:

X_train,X_test,y_train,y_test = train_test_split(final_data_p1.iloc[:,:-1], final_data_p1['meal'], test_size=0.25, random_state=0)

# K-fold cross validation

In [395]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

cv = KFold(n_splits=10, random_state=1, shuffle=True)
# create model
model = LogisticRegression()
model.fit(X_train,y_train)
# evaluate model
scores = cross_val_score(model, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1)
# report performance
print('Accuracy (K-fold): %.3f (with SD: %.3f)' % (np.mean(scores), np.std(scores)))

Accuracy (K-fold): 0.830 (with SD: 0.067)


In [396]:
y_pred = model.predict(X_test)

In [397]:
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [398]:
print('Confusion matrix: \n',confusion_matrix(y_test, y_pred))
print('Precision score: \n',precision_score(y_test, y_pred))

Confusion matrix: 
 [[38  7]
 [11 44]]
Precision score: 
 0.8627450980392157
